In [1]:
# Dependencies and setup
import requests
import matplotlib.pyplot
import pandas as pd
import numpy as np
import time
from scipy.stats import linregress
from pprint import pprint
from datetime import datetime

# Determine city from latitued and longitude
from citipy import citipy

# Get api key from config file, an untracked file in gitignore
from config import api_key

output_files = 'Output/cities.csv'

# Define range of latitudes and longitudes
lat_r=(-45, 45)
lng_r=(-180, 180)

##### Generate List of Cities

In [2]:
# Variable with an empty list to hold cities and lat_long
cities = []
lat_long = []

# Randomize creation of latitude & longitude. Set np size to 1000; change if needed
lat = np.random.uniform(low=-45.00, high=45.00, size=100)
lng = np.random.uniform(low=-180, high = 180, size=100)
lat_long = zip(lat, lng)

# Find nearest city with geo coordinates
for lat_lng in lat_long:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    if city not in cities:
        cities.append(city)

# Validate sufficient number of cities generated        
len(cities)

84

##### API Calls

In [3]:
base_url = 'http://api.openweathermap.org/data/2.5/weather?'
units = 'imperial'

# Setup URL query
query_url = f'{base_url}appid={api_key}&units={units}&q='

# Dictionary of variables where data from api calls will be stored
city_data = {'city_n':[], 'city_lat':[], 'city_lng':[], 'temp':[], 'humdty':[],
             'cloudns':[], 'wndspd':[], 'cntry':[], 'date':[]}

In [4]:
city_ttls = len(cities)
r = 1

print('Retrieving weather data')
print('-' * 30)

for r, city in enumerate(cities):
    try:
        print(f'Retrieving number {r} of {city_ttls}.')
        r+=1
        city = requests.get(query_url + city).json()
        city_data["city_n"].append(city["name"])
        city_data["city_lat"].append(city["coord"]["lat"])
        city_data["city_lng"].append(city["coord"]["lon"])
        city_data["temp"].append(city["main"]["temp"])
        city_data["humdty"].append(city["main"]["humidity"])
        city_data["cloudns"].append(city["clouds"]["all"])
        city_data["wndspd"].append(city["wind"]["speed"])
        city_data["cntry"].append(city["sys"]["country"])
        city_data["date"].append(city["dt"])
        
        # Use timer to delay request to not exceed query limits.
        time.sleep(1.05)
    except:
        print(f'Incomplete record for {city}. Skipping {city}.')
        pass

print(f'Data retrieval completed.')

Retrieving weather data
------------------------------
Retrieving number 0 of 84.
Retrieving number 1 of 84.
Retrieving number 2 of 84.
Retrieving number 3 of 84.
Retrieving number 4 of 84.
Retrieving number 5 of 84.
Retrieving number 6 of 84.
Retrieving number 7 of 84.
Retrieving number 8 of 84.
Retrieving number 9 of 84.
Retrieving number 10 of 84.
Retrieving number 11 of 84.
Retrieving number 12 of 84.
Retrieving number 13 of 84.
Retrieving number 14 of 84.
Retrieving number 15 of 84.
Incomplete record for {'cod': '404', 'message': 'city not found'}. Skipping {'cod': '404', 'message': 'city not found'}.
Retrieving number 16 of 84.
Retrieving number 17 of 84.
Retrieving number 18 of 84.
Incomplete record for {'cod': '404', 'message': 'city not found'}. Skipping {'cod': '404', 'message': 'city not found'}.
Retrieving number 19 of 84.
Retrieving number 20 of 84.
Retrieving number 21 of 84.
Retrieving number 22 of 84.
Retrieving number 23 of 84.
Retrieving number 24 of 84.
Retrieving nu

In [6]:
city_df = pd.DataFrame(city_data) 
city_df.head()

,city_n,city_lat,city_lng,temp,humdty,cloudns,wndspd,cntry,date
0,Sal Rei,16.1833,-22.9167,72.16,88,27,20.80,CV,1622849305
1,Kapaa,22.0752,-159.3190,80.73,74,90,4.00,US,1622849059
2,Lebu,-37.6167,-73.6500,50.94,83,78,5.59,CL,1622849308
3,Mataura,-46.1927,168.8643,49.48,74,100,7.90,NZ,1622849069
4,Mauá,-23.6678,-46.4614,67.46,75,4,2.71,BR,1622849310
